In [1]:
from datetime import date
import json
import os
import re

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import dateutil.parser

In [ ]:
%%time
df = pd.read_csv('../data/scotus_file', usecols = ['author', 'html'])

In [ ]:
print(len(df))
df.dropna(how="any", inplace=True)
print(len(df))

In [ ]:
%%time
def parse_html(raw_html):
    return BeautifulSoup(raw_html, "html.parser").text
df['text'] = df.apply(lambda row: parse_html(row['html']), axis=1)

In [2]:
%%time
df = pd.read_csv('../data/parsed.csv')

CPU times: user 9.72 s, sys: 1.29 s, total: 11 s
Wall time: 11.1 s


In [3]:
%%time
names = 10000 * [None] # names[i] = (first, middle, last)
PEOPLE_ROOT = "../data/people"
for file_name in os.listdir(PEOPLE_ROOT):
    author_idx = int(file_name.split('.')[0])
    with open(os.path.join(PEOPLE_ROOT, file_name)) as f:
        data = json.load(f)
        names[author_idx] = (data['name_first'], data['name_middle'], data['name_last'])
names[0:5]

CPU times: user 870 ms, sys: 542 ms, total: 1.41 s
Wall time: 3.55 s


In [4]:
def get_idx(author_url): # returns int
    return int(author_url.split('/')[-2])
get_idx(df['author'][df.index[0]])

26

In [5]:
%%time
fml = df.apply(lambda row: names[get_idx(row['author'])], axis = 1)
first, middle, last = map(list, zip(*fml))
last[0:5]

CPU times: user 342 ms, sys: 3.08 ms, total: 345 ms
Wall time: 343 ms


In [6]:
df['first'] = first
df['middle'] = middle
df['last'] = last

In [7]:
%%time
# See https://bear.im/code/parsedatetime/docs/index.html for additional documentation
pattern1 = re.compile("Decided.*\n")
pattern2 = re.compile("Term, [0-9]{4}.*\n")
pattern3 = re.compile("\n\[.*[0-9], [0-9]{4}\]\n")
def get_date(text):
    search_res = pattern1.search(text, 0, 500)
    if search_res:
        raw = search_res.group()[7 : -1]
        try:
            return dateutil.parser.parse(raw).date()
        except:
            return None
    
    search_res = pattern2.search(text, 0, 1000)
    if search_res:
        raw = search_res.group()[6:-1]
        raw = "May 9, " + raw
        try:
            return dateutil.parser.parse(raw).date()
        except:
            return None
    
    search_res = pattern3.search(text, 0, 1000)
    if search_res:
        raw = search_res.group()[2:-2]
        try:
            return dateutil.parser.parse(raw).date()
        except:
            return None
    
    lines = text[0 : 500].split('\n')
    for line in lines:
        line = line.replace('[', '')
        line = line.replace(']', '')
        try:
            res = dateutil.parser.parse(line).date()
            return res
        except:
            pass
    
    return None

df['date'] = df.apply(lambda row: get_date(row['text']), axis=1)
#print(get_date(df['text'][1]))

CPU times: user 2.6 s, sys: 5.29 ms, total: 2.61 s
Wall time: 2.61 s


In [8]:
len(df[pd.isna(df.date)])

244

In [9]:
print(len(df))
df.dropna(how="any", inplace=True)
print(len(df))

21337
21093


In [10]:
%%time
df.to_csv('../data/parsedv4.csv')

CPU times: user 28.5 s, sys: 999 ms, total: 29.5 s
Wall time: 30.2 s


In [ ]:
sorted(df['date'])[-1000:]

In [ ]:
list(filter(lambda iy: iy[1].year >=2005, enumerate(df['date'])))

In [ ]:
print(df['date'][df.index[20583]])
print(df['text'][df.index[20583]])

In [ ]:
for i, t in enumerate(df['last']):
    if t == 'Roberts':
        print(i)

In [ ]:
df['text'][df.index[7290]]

In [ ]:
df.index[7290]

In [ ]:
dateutil.parser.parse("[May 17, 1999]")